# **CRDM Beta Simulation (CCB Project): Version 1**

#### - Softmax Choice Model *only*
#### - Simulating across beta [-1.3, 1.31, 2.6/498], obtaining 500 steps (*simulated subjects*)
#### - Alpha constant [0.6]
#### - Gamma constant [1.5]

In [1]:
from glob import glob
import numpy as np
np.seterr(all = "ignore")
import os
import os.path, sys
import pandas as pd
import pylab as plt
import seaborn as sns
sns.set_theme(style = "white", palette = "muted")
from scipy.stats import linregress, shapiro, spearmanr
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

base_proj_dir = "Z:/data/CCB" ## base project directory
save_dir = os.path.join(base_proj_dir, "derivatives/simulations/crdm/simulation1") 
if not os.path.exists(save_dir): ## new Ss
    os.makedirs(save_dir) ## make new Ss save directory

### **Attempt 1**
- Save *everything* to .csv

In [3]:
alpha = 0.6 ## typical risk tolerance
amb_space = [0, 24.0, 50.0, 74.0]
beta_space =  np.round(np.arange(-1.3, 1.31, 2.6/498, dtype=float), 2)
gamma = 1.5 ## typical choice stochasticity
prob_space = [0.13, 0.25, 0.38, 0.5, 0.75]
safe_amt = 5.0 ## safe option amount
safe_prob = 1.0 ## safe option probability is always 100%
value_space = [8, 20, 40, 50] ## possible lottery values


sim_dict = {"sim_sub" : [], "trial" : [], "alpha" : [], "beta" : [], "gamma" : [], 
            "lott_amb" : [], "lott_amt" : [], "lott_prob" : [], "safe_amt" : [], "safe_prob": [],
            "lott_sv" : [], "safe_sv" : [],  "lott_choice_prob" : [], "lott_choice": []}

for sim_sub, beta in enumerate(beta_space): ## 500 simulated subjects
    trial_num = 0
    for x in range(2): ## trials per conditional subgroup
        for lott_amt in value_space:
            for lott_amb in amb_space:
                if lott_amb == 0:
                    for lott_prob in prob_space:
                        trial_num += 1
                        safe_sv = (safe_prob - beta * (lott_amb / 2)) * safe_amt**alpha ## subjective value of safe option
                        lott_sv = (lott_prob - beta * (lott_amb / 2)) * lott_amt**alpha ## subjective value of lottery option
                        pr_lott_choice = 1 / (1 + np.exp(-gamma * (lott_sv - safe_sv))) ## Softmax choice -- probability of choosing lottery option
                        nonsense = np.random.random() ## choice noise
                        if nonsense <= pr_lott_choice: 
                            lott_choice = 1
                        else:
                            lott_choice = 0
                        sim_dict["sim_sub"].append(sim_sub+1)
                        sim_dict["trial"].append(trial_num)
                        sim_dict["alpha"].append(alpha)
                        sim_dict["beta"].append(beta)
                        sim_dict["gamma"].append(gamma)
                        sim_dict["lott_amb"].append(lott_amb)
                        sim_dict["lott_amt"].append(lott_amt)
                        sim_dict["lott_prob"].append(lott_prob)
                        sim_dict["safe_amt"].append(safe_amt)
                        sim_dict["safe_prob"].append(safe_prob)
                        sim_dict["safe_sv"].append(safe_sv)
                        sim_dict["lott_sv"].append(lott_sv)
                        sim_dict["lott_choice_prob"].append(pr_lott_choice)
                        sim_dict["lott_choice"].append(lott_choice)
                else:
                    lott_prob = 0.5
                    trial_num += 1
                    safe_sv = (safe_prob - beta * (lott_amb / 2)) * safe_amt**alpha 
                    lott_sv = (lott_prob - beta * (lott_amb / 2)) * lott_amt**alpha 
                    pr_lott_choice = 1 / (1 + np.exp(-gamma * (lott_sv - safe_sv))) 
                    nonsense = np.random.random()
                    if nonsense <= pr_lott_choice:
                        lott_choice = 1
                    else:
                        lott_choice = 0
                    sim_dict["sim_sub"].append(sim_sub+1)
                    sim_dict["trial"].append(trial_num)
                    sim_dict["alpha"].append(alpha)
                    sim_dict["beta"].append(beta)
                    sim_dict["gamma"].append(gamma)
                    sim_dict["lott_amb"].append(lott_amb)
                    sim_dict["lott_amt"].append(lott_amt)
                    sim_dict["lott_prob"].append(lott_prob)
                    sim_dict["safe_amt"].append(safe_amt)
                    sim_dict["safe_prob"].append(safe_prob)
                    sim_dict["safe_sv"].append(safe_sv)
                    sim_dict["lott_sv"].append(lott_sv)
                    sim_dict["lott_choice_prob"].append(pr_lott_choice)
                    sim_dict["lott_choice"].append(lott_choice)

simulation_df = pd.DataFrame(sim_dict) ## convert dict to DF
filename = os.path.join(save_dir, "simulation1SM_cAvBcG-2trials.csv") #path/filename to save file
simulation_df.to_csv(filename) #save DF to csv

### **Attempt 2**

In [ ]:
sim_file = os.path.join(save_dir, "beta_softmax.csv")
sim_file = pd.read_csv(sim_file)
alpha = [0.6] ## typical risk tolerance
gamma = 1.5 ## typical choice stochasticity
safe_amt = 5.0 ## safe option amount
safe_prob = 1.0 ## safe option probability is always 100%

params = {"sim_sub" : [], "alpha_s": [], "beta_s" : [], "alpha_r" : [], "beta_r": []}

for unique_sub in sim_file["sim_sub"].unique():
    print(unique_sub)
    params["sim_sub"].append(unique_sub)
    curr_sub = sim_file.loc[(sim_file["sim_sub"] == unique_sub)]
    frst_trial = curr_sub.loc[(sim_file["trial"] == 1)]
    params["alpha_s"].append(frst_trial["alpha"].values[0])
    params["beta_s"].append(frst_trial["beta"].values[0])
    unique_ambs = curr_sub["lott_amb"].unique()
    unique_probs = curr_sub["lott_prob"].unique()
    unique_vals = curr_sub["lott_amt"].unique()

    max_ll = None
    best_params = None

    for alpha1 in alpha:
        for beta1 in np.round(np.arange(-1.3, 1.31, 0.05), 2):
            sum_ll = 0 ## score for current alpha and beta
            for this_amb in unique_ambs:
                for this_prob in unique_probs:
                    for this_val in unique_vals:
                        if this_amb != 0 and this_prob != 0.5:
                            continue
                        this_combo = curr_sub.loc[(curr_sub["lott_amb"] == this_amb) & (curr_sub["lott_prob"] == this_prob) & (curr_sub["lott_amt"] == this_val)]
                        choice_sum = sum(this_combo["lott_choice"]) ## how many times did they pick the lottery for this param combo
                        trial_nums = this_combo["lott_choice"].shape[0] ## number of trials in current param combo
                        
                        safe_sv = (safe_prob - beta1 * (this_amb / 2)) * safe_amt**alpha1 ## subjective value of safe option
                        lott_sv = (this_prob - beta1 * (this_amb / 2)) * this_val**alpha1 ## subjective value of lottery option
                        p = 1 / (1 + np.exp(-gamma * (lott_sv - safe_sv))) ## calculating probability of lottery choice for this alpha/beta grid point

                        log_likelihood = (np.log(p) * choice_sum + np.log(1 - p) * (trial_nums - choice_sum)) ## likelihood these params (alpha/beta) produced observed data
                        if log_likelihood == np.inf: ## checking for log(0)
                            log_likelihood = 0 ## don't give weight to this alpha/beta combo
                        sum_ll += log_likelihood ## summing LL
            if max_ll is None or max_ll < sum_ll: ## is max LL really the max?
                max_ll = sum_ll ## if not, make new max LL
                best_params = alpha1, beta1 ## params corresponding to new max LL
                print(best_params)
    params["alpha_r"].append(best_params[0])
    params["beta_r"].append(best_params[1])

params_df = pd.DataFrame(params)
filename2 = os.path.join(save_dir, "simulation1-2SM_cAvBcG-2trials.png") #path/filename to save file
params_df.to_csv(filename2) #save DF to csv